#**Keras tuner for deciding the number of hidden layers and nurons in the nural network.**

In [1]:
pip install keras-tuner

   ---------------------------------------- 0.0/129.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/129.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/129.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/129.1 kB ? eta -:--:--
   ------------ --------------------------- 41.0/129.1 kB 1.9 MB/s eta 0:00:01
   ------------------ -------------------- 61.4/129.1 kB 656.4 kB/s eta 0:00:01
   --------------------------- ----------- 92.2/129.1 kB 744.7 kB/s eta 0:00:01
   --------------------------- ----------- 92.2/129.1 kB 744.7 kB/s eta 0:00:01
   --------------------------- ----------- 92.2/129.1 kB 744.7 kB/s eta 0:00:01
   -------------------------------------- 129.1/129.1 kB 447.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras_tuner.tuners import RandomSearch

In [8]:
dataset = pd.read_csv('Real_Combine.csv')
dataset

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833
...,...,...,...,...,...,...,...,...,...
1088,18.1,24.0,11.2,1015.4,56.0,1.8,15.9,25.9,288.416667
1089,17.8,25.0,10.7,1015.8,54.0,2.3,9.4,22.2,256.833333
1090,13.9,24.5,11.4,1015.0,95.0,0.6,8.7,14.8,169.000000
1091,16.3,23.0,9.8,1016.9,78.0,1.1,7.4,16.5,186.041667


In [9]:
X = dataset.iloc[:,:-1]
Y = dataset.iloc[:,-1]

In [10]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [11]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory= 'project_1',
    project_name = 'Air_quality_index'
)

In [12]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3, random_state=0)
X_train

,T,TM,Tm,SLP,H,VV,V,VM
1068,21.7,30.0,12.2,1014.5,52.0,1.1,6.3,16.5
181,13.6,20.4,7.5,1020.7,67.0,1.1,10.2,40.7
425,32.8,40.6,26.0,1005.8,45.0,1.9,7.2,11.1
1019,28.3,35.8,26.8,1003.9,89.0,1.6,4.3,11.1
9,11.9,18.9,6.3,1020.1,76.0,1.1,8.3,20.6
...,...,...,...,...,...,...,...,...
1033,31.0,35.0,26.8,1004.1,76.0,2.6,6.1,14.8
763,24.9,32.4,15.2,1015.0,57.0,1.9,1.7,9.4
835,34.1,37.2,29.0,998.4,61.0,2.6,11.5,22.2
559,12.3,19.0,8.9,1015.8,91.0,0.5,4.6,9.4


In [14]:
tuner.search(X_train, Y_train,
             epochs=3,
             validation_split=0.2)

Trial 2 Complete [00h 00m 06s]
val_mean_absolute_error: nan

Best val_mean_absolute_error So Far: nan
Total elapsed time: 00h 00m 13s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
12                |9                 |num_layers
384               |128               |units_0
64                |224               |units_1
0.0001            |0.001             |learning_rate
192               |32                |units_2
128               |32                |units_3
96                |32                |units_4
192               |32                |units_5
96                |32                |units_6
128               |32                |units_7
32                |32                |units_8

Epoch 1/3
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 110.8756 - mean_absolute_error: 110.8756 - val_loss: nan - val_mean_absolute_error: nan
Epoch 2/3
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 102.2241 - mean_absolute_error: 102.2241 - val_loss: nan - val_mean_

c:\Users\hp\.conda\envs\PY3_10\lib\site-packages\keras_tuner\src\engine\metrics_tracking.py:111: RuntimeWarning: All-NaN axis encountered
  np.nanmin(values) if self.direction == "min" else np.nanmax(values)


RuntimeError: Number of consecutive failures exceeded the limit of 3.
